# WS24 - Intelligente Informationssysteme

## Block 3: Retrieval Augmented Generation

**Part 5: Advanced Retrieval - Routing**

1. Logical Datasource Routing
2. Semantic Prompt Selection

In [1]:
## FIRST: Initialize the VectorDB and LLM
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma(persist_directory="vector_store", collection_name="lils_blogs", embedding_function=embeddings)
retriever = vectorstore.as_retriever(search_type="similarity") #, search_kwargs={"k": 2})

## LLM with function calling ability
llm = ChatOllama(model="llama3.2:latest", temperature=0)

## 1. Logical Routing: Logical Datasource Routing

Find the right datastore

![Logical Routing](./media/LangChain_Logical_Routing.png "Logical Routing")


see: 
- https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_10_and_11.ipynb


In [2]:
from typing import Literal
from pydantic import BaseModel, Field


# Data model
class RouteQuery(BaseModel):
    """Detect the used programming language."""

    programming_language: Literal["python", "java", "golang"] = Field(
        ...,
        description="Given a user question with some source code as an example identify the programming language.",
    )

# 
# llm.with_structured_output(schema)
# 
# The output schema can be passed in as:
# - an OpenAI function/tool schema,
# - a JSON Schema,
# - a TypedDict class (support added in 0.2.26),
# - or a Pydantic class. If ``schema`` is a Pydantic class then the model output will be a Pydantic instance of that class, 
#   and the model-generated fields will be validated by the Pydantic class. 
#
# Otherwise the model output will be a dict and will not be validated.


structured_llm = llm.with_structured_output(RouteQuery)

In [3]:
# Prompt 
system = """You are an expert in coding and programming in the languages java, python and golang. 

Based on the programming language the question is referring to, return the name of the programming language."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

In [4]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})


In [5]:
result

RouteQuery(programming_language='python')

In [6]:
import inspect

def integer_multiplication(x: int, y: int) -> int:
    return x * y

code_lines = inspect.getsource(integer_multiplication)
print(code_lines)

def integer_multiplication(x: int, y: int) -> int:
    return x * y



In [7]:
question = """ Describe the the following method:

{code_lines}

"""

result = router.invoke(question.format(code_lines=code_lines))

In [8]:
result

RouteQuery(programming_language='python')

In [9]:
question = """ What is wrong with this code example:

public class Main {
  public class void main(String[] args) {
    System.out.println(max(5, 10));  
    }
}

"""
result = router.invoke({"question": question})

In [10]:
result

RouteQuery(programming_language='java')

## 1. Semantic Routing: Prompt Selection

Choose the right prompt

![Semantic Routing](./media/LangChain_Semantic_Routing.png "Semantic Routing")


see: 
- https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_10_and_11.ipynb

In [13]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

restaurant_template = """You are a restaurant with a local menu. You serve local specialities.
Special of today is:
- Flädlesuppe
- Rahmhackbraten mit Champigion, Spätzle und Salat
- Sauerbraten

You are great at answering questions about the special of the day. \

Here is a question:
{query}
"""


# Embed prompts
prompt_templates = [physics_template, math_template, restaurant_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    if most_similar == math_template:
        print("Using MATH")  
    elif most_similar == physics_template:
        print("Using PHYSICS") 
    else:
        print("Beeing a RESTAURANT")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)


In [14]:
print(chain.invoke("What's a matrix"))

Using MATH
A matrix! A fundamental concept in linear algebra and mathematics.

To break it down, let's start with the basics:

**Definition:** A matrix is a rectangular array of numbers, symbols, or expressions, arranged in rows and columns. It's often represented as a set of ordered pairs (a, b), where 'a' represents an element in the first row and column, 'b' represents an element in the second row and column, and so on.

**Components:**

1. **Rows:** A matrix has multiple rows, which are horizontal lines that contain elements.
2. **Columns:** A matrix also has multiple columns, which are vertical lines that contain elements.
3. **Elements:** Each element is a single number or symbol within the matrix.

**Types of Matrices:**

1. **Square Matrix:** A square matrix has the same number of rows and columns.
2. **Rectangular Matrix:** A rectangular matrix has more rows than columns (or vice versa).
3. **Diagonal Matrix:** A diagonal matrix has non-zero elements only on its main diagonal.

In [15]:
print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole! One of the most fascinating and mysterious objects in the universe.

A black hole is essentially a region in space where the gravitational pull is so strong that nothing, including light, can escape. It's formed when a massive star collapses in on itself and its gravity becomes so strong that it warps the fabric of spacetime around it.

Imagine spacetime as a trampoline: if you place a heavy object, like a bowling ball, on the trampoline, it will warp and curve, creating a depression. Now imagine taking that bowling ball and making it infinitely dense and heavy – that's roughly what happens in a black hole. The gravity is so strong that it creates a boundary called the event horizon, which marks the point of no return.

Once something crosses the event horizon, it's trapped by the black hole's gravity and can't escape. That's why black holes are invisible to us, as not even light can escape to reach our eyes.

Black holes come in different sizes, ranging fro

In [16]:
print(chain.invoke("What's Rahmkuchen"))

Beeing a RESTAURANT
Rahmkuchen! That's a classic local specialty from our region. Rahmkuchen is a traditional German bread that's made with fermented rye flour, water, salt, and sometimes caraway seeds or other spices. The fermentation process gives it a distinctive sour taste and a dense, chewy texture.

In our restaurant, we serve Rahmkuchen as a side dish to many of our local specialties, including the Rahmhackbraten you saw on our menu today. It's often served warm, sliced into thick pieces, and is a perfect accompaniment to the rich flavors of the braised beef and vegetables in the Rahmhackbraten.

Would you like to try some Rahmkuchen with your meal today?
